# Modelo para prever a demanda por cimento

Versão refatorada

Nesse notebook carregamos os dados de `TCC/dados/anualizados/csv/` e implementamos uma rede feed forward simples para prever a demanda por cimento de acordo com o estado

Definimos um range de tempo  de janeiro do ano 1991 até dezembro de 2019, adotamos a variável `anomes` para indicar tempo: `199101 <= anomes <= 201912`

Supomos, sempre, que os estados estão ordenado sem ordem alfabética nas tabelas. Além disso, supomos que a primeira coluna da tabela indica a medição janeiro de 1991 (anomes = 199101) e a última, dezembro de 2019 (anomes = 201912), com frequência mensal.

In [1]:
# devolve um array com os valores de ano e mes considerados no modelo
# de janeiro de ANO_INICIAL=1991 ate dezembro de ANO_FINAL=2019
def get_anomes ():

    anos = [str(i) for i in range(ANO_INICIAL, ANO_FINAL+1)]

    # colocamos os 0 a esquerda se precisar
    meses = ["{:02d}".format(i) for i in range(1,13)] 

    anomes = []

    for ano in anos:
        anomes += [str(ano) + i for i in meses]

    return anomes

In [2]:
# funcao que pega um pandas dataframe e coloca os valor 
# no array que sera usado como entrada para o  modelo
def df_to_list(indicador, array : list):

    anomes = get_anomes()
  
    index = 0 

    for coluna in anomes:

        for linha in indicador[coluna]:

            array[index] = array[index] + [linha]
            index += 1
    
    return array

In [3]:
# Carrega os dados de consumo de cimento 
# do PATH considerando a VAR de estabilidade
# devolve uma lista com a tendencia de demanda
def get_consumo (path : str, var : float) -> list :

    import pandas as pd

    anomes = get_anomes()

    consumo = pd.read_csv(path + "consumo.csv")

    # para pegarmos a tendencia de consumo do primeiro mes
    #  consultamos o mes anterior
    anomes = ["199012"] + anomes

    tendencia = []

    # anomes de 19901 ate 201912
    for index  in range(1,len(anomes)):

        # estados...
        for estado in consumo.index:

            tend = 1 - (consumo[anomes[index]][estado]/consumo[anomes[index - 1]][estado])

            # queda
            if tend < - var:
                tendencia += [[1., 0., 0.]]

            # estabilidade
            elif tend <= var:
                tendencia += [[0., 1., 0.]]

            # aumento
            else:
                tendencia += [[0., 0., 1.]]

    return tendencia

In [4]:
# Lê os arquivos csv presentes em 'indicadores' que se encontram no path
# definido na variavel path e carrega no df de entrada, que, na verdade,
# é uma lista
def get_df (path : str, indicadores : list) -> list:

    import pandas as pd

    df = [[i] for i in range(1,28)] * len(get_anomes())

    for ind in indicadores:

      print(f"Carregando o indicador: {ind[:-3]}")
      df = df_to_list(pd.read_csv(path + ind), df)

    return df

In [35]:
### Constantes que definimos
ANO_INICIAL = 1991
ANO_FINAL = 2019
PATH = "../dados/anualizados/csv/"
VAR = 0.01
INDICADORES = [
    "consumo.csv",
    "desemprego.csv",
    "estoque.csv",
    "idh_educacao.csv",
    "idh_renda.csv",
    "idh_saude.csv",
    "IGP.csv",
    "INCC.csv",
    "IPCA.csv",
    "NFSP.csv",
    "pib_construcao.csv",
    "pib_per_capita.csv",
    "pib_precos_constantes.csv",
    "pib_precos_mercado_corrente.csv",
    "populacao.csv",
    "SELIC.csv"]

QUEDA=-1
ESTAB=0
AUMENTO=1

In [6]:
# Dados do consumo 
# from load_data import get_consumo 
tendencia = get_consumo(PATH, VAR)

In [19]:
# Dados de entrada 
# from load_data import get_df 
df = get_df(PATH, INDICADORES)

Carregando o indicador: consumo.
Carregando o indicador: desemprego.
Carregando o indicador: estoque.
Carregando o indicador: idh_educacao.
Carregando o indicador: idh_renda.
Carregando o indicador: idh_saude.
Carregando o indicador: IGP.
Carregando o indicador: INCC.
Carregando o indicador: IPCA.
Carregando o indicador: NFSP.
Carregando o indicador: pib_construcao.
Carregando o indicador: pib_per_capita.
Carregando o indicador: pib_precos_constantes.
Carregando o indicador: pib_precos_mercado_corrente.
Carregando o indicador: populacao.
Carregando o indicador: SELIC.


## Treinamento do modelo

Vamos treinar uma rede neural utilizando a biblioteca TensorFlow

Dividimos os dados em:
- 70%: treino
- 30%: teste

In [21]:
# Importando as libs

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [78]:
# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tendencia,test_size=0.3)

In [79]:
# vamos definir o modelo
# Sequential
model = keras.Sequential()
model.add(Dense(17, input_shape=(17,), activation="softmax"))
model.add(Dense(3, activation="softmax"))

In [80]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [81]:
model.fit(X_train, y_train, epochs=8) 

Epoch 1/8
206/206 [==============================] - 1s 1ms/step - loss: 1.0519 - categorical_accuracy: 0.5025
Epoch 2/8
206/206 [==============================] - 0s 1ms/step - loss: 1.0352 - categorical_accuracy: 0.5025
Epoch 3/8
206/206 [==============================] - 0s 1ms/step - loss: 1.0334 - categorical_accuracy: 0.5025
Epoch 4/8
206/206 [==============================] - 0s 1ms/step - loss: 1.0329 - categorical_accuracy: 0.5025
Epoch 5/8
206/206 [==============================] - 0s 1ms/step - loss: 1.0325 - categorical_accuracy: 0.5025
Epoch 6/8
206/206 [==============================] - 0s 1ms/step - loss: 1.0323 - categorical_accuracy: 0.5025
Epoch 7/8
206/206 [==============================] - 0s 1ms/step - loss: 1.0321 - categorical_accuracy: 0.5025
Epoch 8/8
206/206 [==============================] - 0s 1ms/step - loss: 1.0318 - categorical_accuracy: 0.5025


In [82]:
# Avaliacao
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

89/89 - 0s - loss: 1.0376 - categorical_accuracy: 0.4956 - 226ms/epoch - 3ms/step


In [83]:
# previsao
predictions = model.predict(X_test)

89/89 [==============================] - 0s 958us/step


In [84]:
# transforma dado de [0.85, 0.10,0.05]
# em [-1]
def processa_tendencia (dado):

  lis = []

  for data in dado:

    res = data.index(max(data))
    # Queda
    if res == 0:
      lis += [QUEDA]
    # Estabilidade
    elif res == 1:
      lis += [ESTAB]
    # Aumento
    else:
      lis += [AUMENTO]

  return lis

In [85]:
p = [list(i) for i in predictions]

tend_pred = processa_tendencia(p)
tend_real = processa_tendencia(y_test)

In [86]:
# calcula os acertos do modelo por categoria
def get_accuracy (pred, real):

    acertos = 0
    erros = 0
    estab = 0
    queda = 0
    aumento = 0
    acertos_queda = 0
    acertos_estab = 0
    acertos_aumento = 0

    for i in range(len(pred)):

        if real[i] == QUEDA:

            queda += 1
            
            if pred[i] == QUEDA:
            
                acertos += 1
                acertos_queda += 1
            
            else: 

                erros += 1 

        elif real[i] == ESTAB:
            
            estab += 1
            
            if pred[i] == ESTAB:
            
                acertos += 1
                acertos_estab += 1

            else: 

                erros += 1 

        else:
        
            aumento += 1
            
            if pred[i] == AUMENTO:
            
                acertos += 1
                acertos_aumento += 1

            else: 

                erros += 1  

    print("Acuracia geral obtida: ")
    print(f"Acertos: {acertos} -> {100*acertos/len(pred):.2f}%")
    print(f"Erros: {erros} -> {100*erros/len(pred):.2f}%")
    print("=============================================")

    print("Analisando as quedas:")
    print(f"Numero de quedas: {queda} -> {100*queda/len(pred):.2f}%")
    print(f"Acertos relacao ao total de quedas: {acertos_queda} -> {100*acertos_queda/queda:.2f}%")
    print("=============================================")

    print("Analisando as estabilidades:")
    print(f"Numero de estabilidades: {estab} -> {100*estab/len(pred):.2f}%")
    print(f"Acertos relacao ao total de estabilidades: {acertos_estab} -> {100*acertos_estab/estab:.2f}%")
    print("=============================================")

    print("Analisando os  aumentos:")
    print(f"Numero de aumentos: {aumento} -> {100*aumento/len(pred):.2f}%")
    print(f"Acertos relacao ao total de aumentos: {acertos_aumento} -> {100*acertos_aumento/aumento:.2f}%")
    print("=============================================")


In [87]:
get_accuracy(tend_pred, tend_real)

Acuracia geral obtida: 
Acertos: 1397 -> 49.56%
Erros: 1422 -> 50.44%
Analisando as quedas:
Numero de quedas: 804 -> 28.52%
Acertos relacao ao total de quedas: 127 -> 15.80%
Analisando as estabilidades:
Numero de estabilidades: 1371 -> 48.63%
Acertos relacao ao total de estabilidades: 1258 -> 91.76%
Analisando os  aumentos:
Numero de aumentos: 644 -> 22.84%
Acertos relacao ao total de aumentos: 12 -> 1.86%


In [41]:
data = processa_tendencia(tendencia)

queda = 0
estab = 0
aumento = 0

for i in data:

    if i == QUEDA:
        queda += 1

    elif i == ESTAB:
        estab += 1
    else:
        aumento += 1

print (f"Quedas: {queda} -> {100*queda/(len(data)):.2f}%")
print (f"Estab: {estab} -> {100*estab/(len(data)):.2f}%")
print (f"Aumento: {aumento} -> {100*aumento/(len(data)):.2f}%")

Quedas: 2709 -> 28.83%
Estab: 4589 -> 48.84%
Aumento: 2098 -> 22.33%
